In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pip install librosa soundfile numpy sklearn pyaudio

  Using cached https://files.pythonhosted.org/packages/ab/42/b4f04721c5c5bfc196ce156b3c768998ef8c0ae3654ed29ea5020c749a6b/PyAudio-0.2.11.tar.gz
  ERROR: Failed building wheel for pyaudio
  Running setup.py clean for pyaudio
Failed to build pyaudio
    Running setup.py install for pyaudio ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-6096u1yi/pyaudio/setup.py'"'"'; __file__='"'"'/tmp/pip-install-6096u1yi/pyaudio/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-beq_50ey/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.


In [3]:
!pip install soundfile

In [4]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import os
import sys
import IPython.display as ipd

In [5]:
dataset_path = '/content/drive/MyDrive/Miniproject/DATASET/English'

In [6]:
list_dataset=os.listdir(dataset_path)

In [7]:
dataset=pd.DataFrame(columns=['feature'])
n_dataset=pd.DataFrame(columns=['label'])
ct=0
counter=0

In [8]:
dataset_path = '/content/drive/MyDrive/Miniproject/DATASET/English'

In [9]:
length=len(list_dataset)

for i in range(0,length):
    k=os.listdir(dataset_path+'/'+list_dataset[i])
    j=len(k)
    for u in range(0,j):
        z=list_dataset[i]
        if(z[4]=='h' or z[4]=='H' or z[6]=='h' or z[6]=='H'):
            n_dataset.loc[ct]='Happy'
        elif(z[4]=='a' or z[4]=='A' or z[6]=='a' or z[6]=='A'):
            n_dataset.loc[ct]='Angry'
        elif(z[4]=='n' or z[4]=='N' or z[6]=='n' or z[6]=='N'):
            n_dataset.loc[ct]='Neutral'
        elif(z[4]=='s' or z[4]=='S' or z[6]=='s' or z[6]=='S'):
            n_dataset.loc[ct]='Sad'
        
        ct=ct+1

        X,sample_rate=librosa.load(dataset_path+'/'+list_dataset[i]+'/'+k[u],res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
        sample_rate=np.array(sample_rate)
        mfcc=librosa.feature.mfcc(y=X,sr=sample_rate,n_mfcc=13)
        mfcc=np.mean(mfcc,axis=0)
        dataset.loc[counter]=[mfcc]
        counter=counter+1

In [10]:
dataset = pd.concat([n_dataset,pd.DataFrame(dataset['feature'].values.tolist())],axis=1)

In [11]:
dataset=dataset.fillna(0)

In [12]:
y=dataset.iloc[:,0]
X=dataset.iloc[:,1:218]

In [13]:
dataset=dataset.fillna(0)

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)

In [16]:
mean=np.mean(X_train,axis=0)
std=np.std(X_train,axis=0)

X_train=(X_train-mean)/std
X_test=(X_test-mean)/std

In [17]:
# Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [18]:

# Train the model
model.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [19]:
# Predict for the test set
y_pred=model.predict(X_test)

In [20]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       Angry       0.63      0.62      0.63       269
       Happy       0.60      0.59      0.59       261
     Neutral       0.60      0.61      0.61       270
         Sad       0.64      0.65      0.65       285

    accuracy                           0.62      1085
   macro avg       0.62      0.62      0.62      1085
weighted avg       0.62      0.62      0.62      1085



In [21]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test,y_pred)
print (matrix)

[[168  35  36  30]
 [ 39 153  36  33]
 [ 27  38 165  40]
 [ 34  29  37 185]]


In [22]:
# Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
# Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 61.84%
